In [1]:
"""
On Shrek Docker Only
"""
import fnmatch
import pandas as pd
import shutil
import os
import sys
from ruamel.yaml import YAML
import yaml
os.environ["DLClight"]="True"
import deeplabcut
import glob

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
project = 'Han_202012'
experimenter = 'Joe'
video_project_folder = '/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-04'
path_config_file = video_project_folder + '/config.yaml'
linux_config_file = video_project_folder + '/config_linux.yaml'
shutil.copyfile(path_config_file,linux_config_file)

# convert config file to a linux friendly version
with open(path_config_file) as file:
    orig_config = yaml.load(file, Loader=yaml.FullLoader)

# change project_path to video_main_folder
orig_config['project_path'] = video_project_folder

# change video_sets
# so we need to build a new dictionary with the video filenames as keys and the crop as entries
videos = glob.glob(video_project_folder + r'/videos/*.avi')
x=orig_config['video_sets']
x_keys = list(x.keys())
video_dict = {}
for i in range(len(videos)):
    video_dict[videos[i]] = x[x_keys[i]]
    
orig_config['video_sets'] = video_dict

# save config as config_linux and point path_config_file to it
with open(linux_config_file,'w') as file:
    yaml.dump(orig_config,file)

In [3]:
deeplabcut.create_training_dataset(linux_config_file,userfeedback=False)

It appears that the images were labeled on a Windows system, but you are currently trying to create a training set on a Unix system. 
 In this case the paths should be converted. Do you want to proceed with the conversion?
yes/noyes
Annotation data converted to unix format...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([ 8, 12,  1, 61, 11,  9, 29, 21, 38, 20, 48, 23, 40, 28, 34,  4, 54,
          39, 33, 22, 59, 63, 55, 51,  0, 49, 58, 62, 24, 17, 41, 15, 27,  6,
          18,  5, 32, 35, 26, 44, 10, 37, 25, 47,  7, 13, 60, 52, 43, 53, 19,
          46, 30, 31, 16, 42, 50, 36,  3, 57]), array([14, 45,  2, 56])))]

In [ ]:
deeplabcut.train_network(linux_config_file)
# deeplabcut.train_network(config_path,shuffle=1,trainingsetindex=0,gputouse=None,max_snapshots_to_keep=5,autotune=False,displayiters=100,saveiters=15000, maxiters=30000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12]],
 'all_joints_names': ['shoulder',
                      'arm1',
                      'arm2',
                      'elbow1',
                      'elbow2',
                      'wrist1',
                      'wrist2',
                      'hand1',
                      'hand2',
                      'hand3',
                      'pointX',
                      'pointY',
                      'pointZ'],
 'batch_size': 1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Han_202012Dec4/Han_202012_Joe95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/ho

Selecting single-animal trainer
Starting with imgaug pose-dataset loader (=default).
Batch Size is 1
Initializing ResNet
Loading ImageNet-pretrained resnet_50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-04/dlc-models/iteration-0/Han_202012Dec4-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_

iteration: 1000 loss: 0.0269 lr: 0.005


In [ ]:
deeplabcut.analyze_videos(linux_config_file,videos, save_as_csv=True)

In [ ]:
deeplabcut.filterpredictions(linux_config_file,videos,filtertype='spline')
deeplabcut.plot_trajectories(linux_config_file,videos)
deeplabcut.create_labeled_video(path_config_file,videos,filtered=True)

In [ ]:
# move files from Shrek to server

server_folder_loc = "/mnt/L_MillerLab/data/"
shrek_folder_loc = video_project_folder + "/training-datasets/"

cp_command = "cp -r " + shrek_folder_loc + " " + server_folder_loc

shutil.copytree()

In [ ]:
cp_command